In [3]:
#모듈호출
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import os

In [4]:
target_date = "2015-07" #예측 날짜
seed_word_dict = {"hawkish": ['높', '팽창', '인상', '매파', '성장', '투기_억제', '상승',  '인플레이션_압력', '증가', '위험_선호', '상회', '물가_상승', '과열', '금리_상승', '확장', '상방_압력', '긴축', '변동성_감소', '흑자', '채권_가격_하락', '견조', '요금_인상', '낙관', '부동산_가격_상승', '상향']
                
        
                ,"doveish":["낯", "축", "인하", "비둘기", "둔화", "악화", "하락", "회복_못하", "감소", "위험_회피", "하회", "물가_하락", "위축", "금리_하락", "침체", "하방_압력", "완화", "변동성_확대", "적자", "채권_가격_상승", "부진", "요금_인하", "비관", "부동산_가격_하락", "하향"]}


train_df = pd.read_csv("./train_df.csv")

In [5]:
# 이걸로 테스트
train_list = train_df['words'].tolist()
# 넘파이 코사인 유사도 계산
def cos_sim(A, B):
    return dot(A, B)/(norm(A)*norm(B))

def ngram2vec_train(train_list):
    X_train = train_list

    # 2. 특성 추출 (형태소 분석 및 N-그램)
    vocab = [token.split() for token in X_train]

    # 3. N-그램 추출
    phrases = Phrases([x for x in vocab], min_count=1, threshold=1.1)
    ngram_X_train = [' '.join(phrases[x]) for x in vocab]
    New_test_ngrams = ngram_X_train

    # 4. Ngram2Vec 모델 학습
    model = Word2Vec([x.split() for x in New_test_ngrams], vector_size=300, window=5, min_count=25)
    return model,phrases
model,phrases = ngram2vec_train(train_list)
print("모델 생성 하였습니다")

# 극성 기준 사전
doveish_word_list = []
hawkish_word_list = []
for key in seed_word_dict['doveish']:
    try:
        target = model.wv[key]
        doveish_word_list.append(key)
    except:
        pass

for key in seed_word_dict['hawkish']:
    try:
        target = model.wv[key]
        hawkish_word_list.append(key)
    except:
        pass
model_keys = model.wv.index_to_key
print("극성 기준 사전 생성 하였습니다")

# 카운팅할 극성 워드 생성
Word_Emotion_dict = {"doveish":[],"hawkish":[]}
for key in model_keys:
    doveish_score = 0
    hawkish_score = 0
    target = model.wv[key]
    for word in doveish_word_list:
        doveish_target = model.wv[word]
        doveish_score += cos_sim(target, doveish_target)
    for word in hawkish_word_list:
        hawkish_target = model.wv[word]
        hawkish_score += cos_sim(target, hawkish_target)
    if doveish_score / len(doveish_word_list) > hawkish_score / len(hawkish_word_list):
        #print(f"{key} 해당 키는 비둘기적입니다")
        Word_Emotion_dict["doveish"].append(key)
    elif doveish_score / len(doveish_word_list) == hawkish_score / len(hawkish_word_list):
        print(f"{key} 해당 키는 중립적입니다")
    elif doveish_score / len(doveish_word_list) < hawkish_score / len(hawkish_word_list):
        #print(f"{key} 해당 키는 매파적입니다")
        Word_Emotion_dict["hawkish"].append(key)

print("학습 단어 극성 분류 사전 생성 하였습니다")
print("새로운 문서에 대한 스코어 계산을 시작합니다")

start_date = "2013-05-09"
def select_MPB(target_date):
    def date_range(start, end):
        start = datetime.strptime(start, "%Y-%m-%d")
        end = datetime.strptime(end, "%Y-%m")
        before_one_day = end - relativedelta(days=30)
        dates = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((before_one_day-start).days+1)]
        return dates
    dates = date_range(start_date, target_date)

    path = "./MPB_crawling_dataset"
    file_list = []
    if os.path.isdir(path):
        text_list = os.listdir(path)
        for i in text_list:
            if i[-3:] == "txt":
                target_file = i[:-4]
                if target_file in dates:
                    file_list.append(i)
    MPB_file = file_list[-1]                
    f = open(f"{path}/{MPB_file}", 'r')
    file_text = f.readline()
    return file_text

file_text = select_MPB(target_date)
target_text = (phrases[file_text.split()])
print("target_text 생성")

# 새로운 문서에 대한 스코어 계산
new_word_doveish_score = 0
new_word_hawkish_score = 0
for token in target_text:
    if token in Word_Emotion_dict['doveish']:
        new_word_doveish_score += 1 
    elif token in Word_Emotion_dict['hawkish']:
        new_word_hawkish_score += 1
print(new_word_doveish_score, new_word_hawkish_score)
if new_word_doveish_score >  new_word_hawkish_score:
    print(f"{target_date} 해당 날짜는 비둘기적입니다")
    #return f"{target_date} 해당 날짜는 비둘기적입니다"
elif new_word_doveish_score ==  new_word_hawkish_score:
    print("{target_date} 해당 날짜는 중립 성향 입니다")
    #return f"{target_date} 해당 날짜는 중립 성향 입니다"
elif new_word_doveish_score <  new_word_hawkish_score:
    print(f"{target_date} 해당 날짜는 매파적입니다")
    #return f"{target_date} 해당 날짜는 매파적입니다"


모델 생성 하였습니다
극성 기준 사전 생성 하였습니다
학습 단어 극성 분류 사전 생성 하였습니다
새로운 문서에 대한 스코어 계산을 시작합니다
target_text 생성
732 1155
2015-07 해당 날짜는 매파적입니다


In [6]:
#정확도 계산
# 콜금리기준으로 맞춘 스코어와 못맞춘 스코어를 계산합니다.

#1 극성 페이지 추출
def make_Emotion_dict():
    f = open(f"./call_rate_months.csv", 'r')
    file_lines = f.readlines()
    date = file_lines[0].split(",")
    rate = file_lines[1].split(",")
    date = date[:-4]
    rate = rate[:-4]
    date_dict = {date[idx]: rate[idx] for idx in range(len(rate))}
    target_date_dict = {}
    for i in date_dict:
        temp = i.replace("/",".")
        target_date_dict[temp] = date_dict[i]
    key_list = [key for key, value in target_date_dict.items()]
    value_list = [value for key, value in target_date_dict.items()]
    Emotion_dict = {"doveish":[], "hawkish":[]}
    for idx in range(len(key_list)-1):
        target_date = key_list[idx]
        target_rate = value_list[idx]
        compare_date = key_list[idx+1]
        compare_rate = value_list[idx+1]
        threshold_temp = float(target_rate) - float(compare_rate)
        threshold_temp = (threshold_temp**2)**0.5
        if threshold_temp > 0.001:
            if target_rate > compare_rate:
                Emotion_dict["doveish"] += [[target_date, compare_date]]
            else:
                Emotion_dict["hawkish"] += [[target_date, compare_date]]
    return Emotion_dict

Emotion_dict = make_Emotion_dict()
Emotion_dict

MPB_dict = {"doveish":[], "hawkish":[]}
for Emotions in Emotion_dict:
    if Emotions == "doveish":
        Emotion = list(Emotion_dict[Emotions])
        for date in Emotion:
            start = date[0]
            end = date[1]
            start_time = datetime.strptime(start, "%Y.%m")
            end_time_1 = datetime.strptime(end, "%Y.%m")
            end_time_2 = end_time_1 - timedelta(days=1)
            for day in range((end_time_2-start_time).days+1):
                target = (start_time + timedelta(days=day)).strftime("%Y-%m-%d")
                MPB_Path = f"./MPB_crawling_dataset/{target}.txt"
                file_name = f"{target}.txt"
                if os.path.isfile(MPB_Path):
                     MPB_dict["doveish"].append(file_name)
                     
    else:
        Emotion = list(Emotion_dict[Emotions])
        for date in Emotion:
            start = date[0]
            end = date[1]
            start_time = datetime.strptime(start, "%Y.%m")
            end_time_1 = datetime.strptime(end, "%Y.%m")
            end_time_2 = end_time_1 - timedelta(days=1)
            for day in range((end_time_2-start_time).days+1):
                target = (start_time + timedelta(days=day)).strftime("%Y-%m-%d")
                MPB_Path = f"./MPB_crawling_dataset/{target}.txt"
                file_name = f"{target}.txt"
                if os.path.isfile(MPB_Path):
                    MPB_dict["hawkish"].append(file_name)

total_len = 0
total = {"answer" : 0, "wrong answer" : 0}

for i in MPB_dict['doveish']:
    total_len += 1
    target_MPB_Path = f"./MPB_crawling_dataset/{i}"
    f = open(f"{target_MPB_Path}", 'r')
    accuracy_text = f.readline()
    accuracy_target = (phrases[accuracy_text.split()])
    new_word_doveish_score = 0
    new_word_hawkish_score = 0
    for token in accuracy_target:
        if token in Word_Emotion_dict['doveish']:
            new_word_doveish_score += 1 
        elif token in Word_Emotion_dict['hawkish']:
            new_word_hawkish_score += 1
    print(f"비둘기 점수: {new_word_doveish_score}, 매파 점수: {new_word_hawkish_score}")
    if new_word_doveish_score >  new_word_hawkish_score:
        total["answer"] += 1
    else:
        total["wrong answer"] += 1

for i in MPB_dict["hawkish"]:
    total_len += 1
    target_MPB_Path = f"./MPB_crawling_dataset/{i}"
    f = open(f"{target_MPB_Path}", 'r')
    accuracy_text = f.readline()
    accuracy_target = (phrases[accuracy_text.split()])
    new_word_doveish_score = 0
    new_word_hawkish_score = 0
    for token in accuracy_target:
        if token in Word_Emotion_dict['doveish']:
            new_word_doveish_score += 1 
        elif token in Word_Emotion_dict['hawkish']:
            new_word_hawkish_score += 1
    print(f"비둘기 점수: {new_word_doveish_score}, 매파 점수: {new_word_hawkish_score}")
    if new_word_doveish_score <  new_word_hawkish_score:
        total["answer"] += 1
    else:
        total["wrong answer"] += 1

print(f"정답률 : {total['answer'] / total_len}, 오답률 :{total['wrong answer'] / total_len}")
                    

비둘기 점수: 838, 매파 점수: 1423
비둘기 점수: 661, 매파 점수: 1284
비둘기 점수: 9, 매파 점수: 17
비둘기 점수: 876, 매파 점수: 1525
비둘기 점수: 1018, 매파 점수: 1780
비둘기 점수: 880, 매파 점수: 1481
비둘기 점수: 1093, 매파 점수: 2049
비둘기 점수: 32, 매파 점수: 105
비둘기 점수: 1075, 매파 점수: 1983
비둘기 점수: 1041, 매파 점수: 1770
비둘기 점수: 34, 매파 점수: 103
비둘기 점수: 1114, 매파 점수: 1873
비둘기 점수: 54, 매파 점수: 111
비둘기 점수: 882, 매파 점수: 1609
비둘기 점수: 834, 매파 점수: 1638
비둘기 점수: 20, 매파 점수: 33
비둘기 점수: 732, 매파 점수: 1155
비둘기 점수: 901, 매파 점수: 1431
비둘기 점수: 44, 매파 점수: 116
비둘기 점수: 1051, 매파 점수: 1482
비둘기 점수: 54, 매파 점수: 117
비둘기 점수: 880, 매파 점수: 1540
비둘기 점수: 8, 매파 점수: 26
비둘기 점수: 1007, 매파 점수: 1623
비둘기 점수: 11, 매파 점수: 36
비둘기 점수: 665, 매파 점수: 1329
비둘기 점수: 784, 매파 점수: 1387
비둘기 점수: 75, 매파 점수: 171
비둘기 점수: 614, 매파 점수: 1300
비둘기 점수: 48, 매파 점수: 181
비둘기 점수: 655, 매파 점수: 1309
비둘기 점수: 1, 매파 점수: 3
비둘기 점수: 581, 매파 점수: 1042
비둘기 점수: 31, 매파 점수: 74
비둘기 점수: 77, 매파 점수: 201
비둘기 점수: 8, 매파 점수: 27
비둘기 점수: 606, 매파 점수: 1289
비둘기 점수: 16, 매파 점수: 37
비둘기 점수: 108, 매파 점수: 303
비둘기 점수: 696, 매파 점수: 1474
비둘기 점수: 8, 매파 점수: 25
비둘기 점수: 540, 매파 점수

KeyboardInterrupt: 

In [7]:
print(f"정답률 : {total['answer'] / total_len}, 오답률 :{total['wrong answer'] / total_len}")

정답률 : 0.460431654676259, 오답률 :0.5323741007194245
